In [29]:
import json
import pandas as pd
import statsmodels.api as sm
import yfinance as yf
from scipy.optimize import minimize
import math
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [30]:
# 'yyyy-mm-dd'
nifty50 = yf.download('^NSEI', '2023-01-01', '2024-01-01', interval="1d")
nifty50 = nifty50[['Adj Close']]
nifty50.columns = [['Nifty 50']]
nifty50.index = pd.to_datetime(nifty50.index)
nifty50

[*********************100%%**********************]  1 of 1 completed


,Nifty 50
Date,
2023-07-03,19322.550781
2023-07-04,19389.000000
2023-07-05,19398.500000
2023-07-06,19497.300781
2023-07-07,19331.800781
...,...
2023-12-22,21349.400391
2023-12-26,21441.349609
2023-12-27,21654.750000


In [31]:
file_name = 'industryWise_monthWise_stocks_intersection_of_both.json'

with open(file_name, 'r') as file:
    data = json.load(file)

yearWise_dict_stocks_intersection_of_both = {}
for year, dict in data.items():
    yearWise_dict_stocks_intersection_of_both[year] = []
    for industry, list_ in dict.items():
        yearWise_dict_stocks_intersection_of_both[year] += list_

yearWise_dict_stocks_intersection_of_both

{'2023_12': ['BSE.NS',
  'SUZLON.NS',
  'ORIENTCEM.NS',
  'NEULANDLAB.NS',
  'HSCL.NS',
  'NEWGEN.NS',
  'JPPOWER.NS',
  'GMDCLTD.NS',
  'RTNINDIA.NS',
  'CHENNPETRO.NS',
  'LTFOODS.NS',
  'EMIL.NS',
  'JPASSOCIAT.NS',
  'GOKEX.NS',
  'HBLPOWER.NS',
  'KALYANKJIL.NS',
  'DBREALTY.NS',
  'ITI.NS',
  'DBCORP.NS']}

In [32]:
file_name = 'industryWise_monthWise_stocks_no_zero_weightage.json'

with open(file_name, 'r') as file:
    data = json.load(file)

yearWise_dict_stocks_no_zero_weightage = {}
for year, dict in data.items():
    yearWise_dict_stocks_no_zero_weightage[year] = []
    for industry, list_ in dict.items():
        yearWise_dict_stocks_no_zero_weightage[year] += list_

yearWise_dict_stocks_no_zero_weightage

{'2023_12': ['ITI.NS',
  'NEWGEN.NS',
  'HBLPOWER.NS',
  'DBCORP.NS',
  'GOKEX.NS',
  'RTNINDIA.NS',
  'EMIL.NS',
  'LTFOODS.NS',
  'BSE.NS',
  'HSCL.NS',
  'JPPOWER.NS',
  'NEULANDLAB.NS',
  'ORIENTCEM.NS',
  'CHENNPETRO.NS',
  'DBREALTY.NS',
  'JPASSOCIAT.NS',
  'SUZLON.NS',
  'GMDCLTD.NS',
  'KALYANKJIL.NS']}

In [33]:
file_name = 'industryWise_monthWise_stocks_overall_highest_weightage.json'

with open(file_name, 'r') as file:
    data = json.load(file)

yearWise_dict_stocks_overall_highest_weightage = {}
for year, dict in data.items():
    yearWise_dict_stocks_overall_highest_weightage[year] = []
    for industry, list_ in dict.items():
        yearWise_dict_stocks_overall_highest_weightage[year] += list_

yearWise_dict_stocks_overall_highest_weightage

{'2023_12': ['ITI.NS',
  'NEWGEN.NS',
  'HBLPOWER.NS',
  'DBCORP.NS',
  'GOKEX.NS',
  'RTNINDIA.NS',
  'EMIL.NS',
  'LTFOODS.NS',
  'BSE.NS',
  'HSCL.NS',
  'JPPOWER.NS',
  'NEULANDLAB.NS',
  'ORIENTCEM.NS',
  'CHENNPETRO.NS',
  'DBREALTY.NS',
  'JPASSOCIAT.NS',
  'SUZLON.NS',
  'GMDCLTD.NS',
  'KALYANKJIL.NS']}

In [34]:
final_stock_df = pd.read_csv('Final Stock Data.csv', index_col='Date')
final_stock_df.index = pd.to_datetime(final_stock_df.index)
final_stock_df

,360ONE.NS,3MINDIA.NS,ABB.NS,ACC.NS,AGI.NS,AIAENG.NS,APLAPOLLO.NS,AUBANK.NS,AARTIDRUGS.NS,AARTIIND.NS,...,YESBANK.NS,ZFCVINDIA.NS,ZEEL.NS,ZENTEC.NS,ZENSARTECH.NS,ZOMATO.NS,ZYDUSLIFE.NS,ZYDUSWELL.NS,ECLERX.NS,EMUDHRA.NS
Date,,,,,,,,,,,,,,,,,,,,,
2023-07-03,468.526886,27129.734375,4405.655762,1806.601196,573.233032,3142.567871,1324.286743,759.249939,460.850006,502.606598,...,16.350000,12284.312500,180.850006,421.448395,382.826721,75.650002,573.056824,1475.409790,1642.853149,416.049988
2023-07-04,474.879578,26981.560547,4355.170410,1801.998291,559.773865,3123.199707,1308.184937,776.177673,458.649994,489.100769,...,16.799999,12132.475586,182.449997,415.249908,380.494507,74.800003,568.796509,1475.958008,1640.204712,411.549988
2023-07-05,467.738892,27229.531250,4385.032227,1802.344849,564.740356,3167.462646,1308.783081,768.038391,467.250000,484.764923,...,17.150000,12148.507812,183.100006,414.100189,379.551727,74.050003,579.595825,1474.512695,1614.120605,463.049988
2023-07-06,472.515778,27264.554688,4394.569824,1811.099976,566.528259,3201.219971,1325.134155,788.810974,470.600006,484.416077,...,17.799999,12156.099609,189.350006,419.498901,387.044495,74.750000,582.667175,1478.848267,1676.432739,468.899994
2023-07-07,474.682617,26902.242188,4394.170410,1791.500000,571.693420,3194.249512,1313.718384,760.648132,470.700012,470.561371,...,17.450001,12046.599609,206.149994,429.696411,385.208527,73.250000,576.871216,1476.605835,1691.423584,457.549988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,657.900024,31182.000000,4759.000000,2090.750000,832.000000,3574.300049,1588.150024,765.450012,499.500000,598.549988,...,21.049999,15536.599609,267.549988,746.849976,620.049988,128.500000,679.950012,1590.150024,2580.100098,434.850006
2023-12-26,651.500000,31447.599609,4771.350098,2113.550049,844.450012,3590.699951,1588.050049,769.549988,493.950012,633.099976,...,20.950001,15333.200195,264.399994,771.799988,619.150024,125.000000,676.200012,1604.099976,2615.850098,438.850006
2023-12-27,674.450012,36389.949219,4733.600098,2168.550049,844.000000,3610.550049,1582.650024,774.599976,497.700012,645.750000,...,20.950001,15463.599609,266.100006,810.150024,627.250000,127.050003,688.650024,1634.050049,2595.500000,432.200012


In [35]:
if len(final_stock_df) > len(nifty50):
    final_stock_df = final_stock_df[final_stock_df.index.isin(nifty50.index)]
elif len(nifty50) > len(final_stock_df):
    nifty50 = nifty50[nifty50.index.isin(final_stock_df.index)]

final_stock_df = final_stock_df.backfill()
nifty50 = nifty50.backfill()

In [36]:
# Function to perform linear regression and return alpha and beta
def regress_stock_with_index(df, stock_column, nifty50_temp_wr):
    X = nifty50_temp_wr['Nifty 50']
    Y = df[stock_column]
    X = sm.add_constant(X) # adding a constant for intercept
    model = sm.OLS(Y, X).fit()
    return model.params

In [37]:
# Single Index Model - Minimize Variance

def smi_min_variance(index_average_return, index_std_dev, alpha_list, beta_list, expected_returns, num_symbs):

    def get_alpha_beta_total(weights):
        alpha_total = 0
        i = 0
        for alpha in alpha_list:
            alpha_total += alpha*weights[i]
            i += 1

        i = 0
        beta_total = 0
        for beta in beta_list:
            beta_total += beta*weights[i]
            i += 1

        return alpha_total, beta_total

    def daily_pf_rtn(weights):
        return get_alpha_beta_total(weights)[0] + (get_alpha_beta_total(weights)[1] * index_average_return)

    def ann_pf_rtn(weights):
        return ((1+daily_pf_rtn(weights))**52)-1

    def daily_pf_var(weights):
        return (get_alpha_beta_total(weights)[1] * index_std_dev)**2

    def daily_std_dev(weights):
        return get_alpha_beta_total(weights)[1] * index_std_dev
    
    def ann_std_dev(weights):
        daily_std_dev_ = get_alpha_beta_total(weights)[1] * index_std_dev
        return daily_std_dev_*math.sqrt(52)
    
    def objective_function(weights):
        var = daily_pf_var(weights)
        return var
    
    def non_zero(weights):
        count = 0
        for w in weights:
            if round(w,4) != 0.0000:
                count += 1
        return count
    
    def no_above_25(weights):
        for w in weights:
            if round(w,4) >= 0.2500:
                return 0
        return 1

    weights = []
    for i in range(num_symbs):
        weights.append(1/num_symbs)

    # Constraints and bounds
    constraints = (
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
        {'type': 'ineq', 'fun': lambda x: ann_pf_rtn(x) - expected_returns},
        {'type': 'ineq', 'fun': lambda x: non_zero(x) - 8},
        {'type': 'eq', 'fun': lambda x: no_above_25(x) - 1}
    )

    bounds = []
    for i in range(num_symbs):
        bounds.append((0,1))

    # Perform the optimization using the initial guess from before
    opt_result_1 = minimize(
        objective_function,
        weights,
        method='trust-constr',
        bounds=bounds,
        constraints=constraints
    )

    opt_result_2 = minimize(
        objective_function,
        weights,
        method='slsqp',
        bounds=bounds,
        constraints=constraints
    )

    if opt_result_1.fun <= opt_result_2.fun:
        opt_result = opt_result_1
    else:
        opt_result = opt_result_2

    new_weights_temp = list(opt_result.x)
    new_weights = []
    for i in new_weights_temp:
        new_weights.append(round(i,4))

    result_dict = {
        'Optimal Weights': new_weights,
        'Min Variance': opt_result.fun,
        'Weekly Return': daily_pf_rtn(new_weights),
        'Annual Return': ann_pf_rtn(new_weights),
        'Weekly Std Dev': daily_std_dev(new_weights),
        'Annual Std Dev': ann_std_dev(new_weights)
    }
    
    return result_dict

In [38]:
# Single Index Model - Maximize Returns

def smi_max_returns(index_average_return, index_std_dev, alpha_list, beta_list, maximum_risk, num_symbs):

    def get_alpha_beta_total(weights):
        alpha_total = 0
        i = 0
        for alpha in alpha_list:
            alpha_total += alpha*weights[i]
            i += 1

        i = 0
        beta_total = 0
        for beta in beta_list:
            beta_total += beta*weights[i]
            i += 1

        return alpha_total, beta_total

    def daily_pf_rtn(weights):
        return get_alpha_beta_total(weights)[0] + (get_alpha_beta_total(weights)[1] * index_average_return)

    def ann_pf_rtn(weights):
        return ((1+daily_pf_rtn(weights))**52)-1

    def daily_pf_var(weights):
        return (get_alpha_beta_total(weights)[1] * index_std_dev)**2

    def daily_std_dev(weights):
        return get_alpha_beta_total(weights)[1] * index_std_dev
    
    def ann_std_dev(weights):
        daily_std_dev_ = get_alpha_beta_total(weights)[1] * index_std_dev
        return daily_std_dev_*math.sqrt(52)
    
    def objective_function(weights):
        ret = daily_pf_rtn(weights)
        return -ret
    
    def non_zero(weights):
        count = 0
        for w in weights:
            if round(w,4) != 0.0000:
                count += 1
        return count
    
    def no_above_25(weights):
        for w in weights:
            if round(w,4) >= 0.2500:
                return 0
        return 1

    weights = []
    for i in range(num_symbs):
        weights.append(1/num_symbs)

    # Constraints and bounds
    constraints = (
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
        {'type': 'ineq', 'fun': lambda x: maximum_risk - ann_std_dev(x)},
        {'type': 'ineq', 'fun': lambda x: non_zero(x) - 8},
        {'type': 'eq', 'fun': lambda x: no_above_25(x) - 1}
    )
    
    bounds = []
    for i in range(num_symbs):
        bounds.append((0,1))
    
    # Perform the optimization using the initial guess from before
    opt_result_1 = minimize(
        objective_function,
        weights,
        method='trust-constr',
        bounds=bounds,
        constraints=constraints
    )

    opt_result_2 = minimize(
        objective_function,
        weights,
        method='slsqp',
        bounds=bounds,
        constraints=constraints
    )

    if opt_result_1.fun <= opt_result_2.fun:
        opt_result = opt_result_1
    else:
        opt_result = opt_result_2

    new_weights_temp = list(opt_result.x)
    new_weights = []
    for i in new_weights_temp:
        new_weights.append(round(i,4))

    result_dict = {
        'Optimal Weights': new_weights,
        'Min Variance': opt_result.fun,
        'Weekly Return': daily_pf_rtn(new_weights),
        'Annual Return': ann_pf_rtn(new_weights),
        'Weekly Std Dev': daily_std_dev(new_weights),
        'Annual Std Dev': ann_std_dev(new_weights)
    }
    
    return result_dict

In [39]:
expected_returns = 0.2
maximum_risk = 0.4

In [40]:
def top_8_stocks(stock_dict):
    # Flatten the dictionary into a list of tuples
    all_stocks = []
    for key in stock_dict:
        all_stocks.extend(stock_dict[key])

    # Sort the list based on weights (second element of each tuple)
    sorted_stocks = sorted(all_stocks, key=lambda x: x[1], reverse=True)

    # Get the top 8 stocks
    top_8 = sorted_stocks[:8]

    symbs = []
    for i,j in top_8:
        symbs.append(i)

    return symbs

In [41]:
results_stocks_overall_highest_weightage = {}
portfolio_overall_highest_weightage = {}

for year_month, stocks in yearWise_dict_stocks_overall_highest_weightage.items():

    print(year_month)
    year = year_month[:4]
    month = year_month[5:]
    print(year, month)

    historical_price = final_stock_df[stocks]
    historical_price = historical_price[(historical_price.index >= f'{month}-01-{int(year)-5}') & (historical_price.index < f'{month}-01-{year}')]

    nifty50_temp = nifty50[(nifty50.index >= f'{month}-01-{int(year)-5}') & (nifty50.index < f'{month}-01-{year}')]
    nifty50_temp_wr = nifty50_temp.pct_change().dropna()

    weekly_returns = historical_price.pct_change().dropna()
    cov_matrix = weekly_returns.cov()

    symbols = historical_price.columns.to_list()

    results = {}
    stock_list = symbols.copy()

    for stock in stock_list:
        results[stock] = regress_stock_with_index(weekly_returns, stock, nifty50_temp_wr)

    alpha_beta_dict = {}
    alpha_list = []
    beta_list = []

    for stock in stock_list:
        alpha = results[stock]['const']
        beta = results[stock][('Nifty 50',)]
        alpha_list.append(alpha)
        beta_list.append(beta)
        alpha_beta_tuple = (alpha, beta)
        alpha_beta_dict[stock] = alpha_beta_tuple

    index_std_dev = nifty50_temp_wr.std().values[0]
    index_average_return = nifty50_temp_wr.mean().values[0]

    num_symbs = len(historical_price.columns)

    results_dict = smi_min_variance(index_average_return, index_std_dev, alpha_list, beta_list, expected_returns, num_symbs)

    results_stocks_overall_highest_weightage[year_month] = results_dict

    portfolio = []
    for i in range(len(symbols)):
        symbol = symbols[i]
        weight = results_dict['Optimal Weights'][i]
        portfolio.append((symbol, weight))

    portfolio_overall_highest_weightage[year_month] = portfolio

    symbols = top_8_stocks(portfolio_overall_highest_weightage)

    historical_price = final_stock_df[symbols]
    historical_price = historical_price[(historical_price.index >= f'{month}-01-{int(year)-5}') & (historical_price.index < f'{month}-01-{year}')]

    weekly_returns = historical_price.pct_change().dropna()
    cov_matrix = weekly_returns.cov()

    symbols = historical_price.columns.to_list()

    results = {}
    stock_list = symbols.copy()

    for stock in stock_list:
        results[stock] = regress_stock_with_index(weekly_returns, stock, nifty50_temp_wr)

    alpha_beta_dict = {}
    alpha_list = []
    beta_list = []

    for stock in stock_list:
        alpha = results[stock]['const']
        beta = results[stock][('Nifty 50',)]
        alpha_list.append(alpha)
        beta_list.append(beta)
        alpha_beta_tuple = (alpha, beta)
        alpha_beta_dict[stock] = alpha_beta_tuple

    index_std_dev = nifty50_temp_wr.std().values[0]
    index_average_return = nifty50_temp_wr.mean().values[0]

    num_symbs = len(historical_price.columns)

    results_dict = smi_max_returns(index_average_return, index_std_dev, alpha_list, beta_list, maximum_risk, num_symbs)

    results_stocks_overall_highest_weightage[year_month] = results_dict

    portfolio = []
    for i in range(len(symbols)):
        symbol = symbols[i]
        weight = results_dict['Optimal Weights'][i]
        portfolio.append((symbol, weight))

    portfolio_overall_highest_weightage[year_month] = portfolio

2023_12
2023 12


In [42]:
portfolio_overall_highest_weightage

{'2023_12': [('LTFOODS.NS', 0.0345),
  ('EMIL.NS', 0.2499),
  ('DBCORP.NS', 0.0679),
  ('BSE.NS', 0.2499),
  ('SUZLON.NS', 0.1406),
  ('NEULANDLAB.NS', 0.0682),
  ('KALYANKJIL.NS', 0.0925),
  ('NEWGEN.NS', 0.0965)]}

In [43]:
json_file_path = 'portfolio_overall_highest_weightage.json'

with open(json_file_path, 'w') as json_file:
    json.dump(portfolio_overall_highest_weightage, json_file)

In [44]:
results_stocks_no_zero_weightage = {}
portfolio_no_zero_weightage = {}

for year_month, stocks in yearWise_dict_stocks_no_zero_weightage.items():

    year = year_month[:4]

    historical_price = final_stock_df[stocks]
    historical_price = historical_price[(historical_price.index >= f'{month}-01-{int(year)-5}') & (historical_price.index < f'{month}-01-{year}')]

    nifty50_temp = nifty50[(nifty50.index >= f'{month}-01-{int(year)-5}') & (nifty50.index < f'{month}-01-{year}')]
    nifty50_temp_wr = nifty50_temp.pct_change().dropna()

    weekly_returns = historical_price.pct_change().dropna()
    cov_matrix = weekly_returns.cov()

    symbols = historical_price.columns.to_list()

    results = {}
    stock_list = symbols.copy()

    for stock in stock_list:
        results[stock] = regress_stock_with_index(weekly_returns, stock, nifty50_temp_wr)

    alpha_beta_dict = {}
    alpha_list = []
    beta_list = []

    for stock in stock_list:
        alpha = results[stock]['const']
        beta = results[stock][('Nifty 50',)]
        alpha_list.append(alpha)
        beta_list.append(beta)
        alpha_beta_tuple = (alpha, beta)
        alpha_beta_dict[stock] = alpha_beta_tuple

    index_std_dev = nifty50_temp_wr.std().values[0]
    index_average_return = nifty50_temp_wr.mean().values[0]

    num_symbs = len(historical_price.columns)

    results_dict = smi_min_variance(index_average_return, index_std_dev, alpha_list, beta_list, expected_returns, num_symbs)

    results_stocks_no_zero_weightage[year_month] = results_dict

    portfolio = []
    for i in range(len(symbols)):
        symbol = symbols[i]
        weight = results_dict['Optimal Weights'][i]
        portfolio.append((symbol, weight))

    portfolio_no_zero_weightage[year_month] = portfolio

    symbols = top_8_stocks(portfolio_no_zero_weightage)

    historical_price = final_stock_df[symbols]
    historical_price = historical_price[(historical_price.index >= f'{month}-01-{int(year)-5}') & (historical_price.index < f'{month}-01-{year}')]

    weekly_returns = historical_price.pct_change().dropna()
    cov_matrix = weekly_returns.cov()

    symbols = historical_price.columns.to_list()

    results = {}
    stock_list = symbols.copy()

    for stock in stock_list:
        results[stock] = regress_stock_with_index(weekly_returns, stock, nifty50_temp_wr)

    alpha_beta_dict = {}
    alpha_list = []
    beta_list = []

    for stock in stock_list:
        alpha = results[stock]['const']
        beta = results[stock][('Nifty 50',)]
        alpha_list.append(alpha)
        beta_list.append(beta)
        alpha_beta_tuple = (alpha, beta)
        alpha_beta_dict[stock] = alpha_beta_tuple

    index_std_dev = nifty50_temp_wr.std().values[0]
    index_average_return = nifty50_temp_wr.mean().values[0]

    num_symbs = len(historical_price.columns)

    results_dict = smi_max_returns(index_average_return, index_std_dev, alpha_list, beta_list, maximum_risk, num_symbs)

    results_stocks_no_zero_weightage[year_month] = results_dict

    portfolio = []
    for i in range(len(symbols)):
        symbol = symbols[i]
        weight = results_dict['Optimal Weights'][i]
        portfolio.append((symbol, weight))

    portfolio_no_zero_weightage[year_month] = portfolio

In [45]:
portfolio_no_zero_weightage

{'2023_12': [('LTFOODS.NS', 0.0345),
  ('EMIL.NS', 0.2499),
  ('DBCORP.NS', 0.0679),
  ('BSE.NS', 0.2499),
  ('SUZLON.NS', 0.1406),
  ('NEULANDLAB.NS', 0.0682),
  ('KALYANKJIL.NS', 0.0925),
  ('NEWGEN.NS', 0.0965)]}

In [46]:
json_file_path = 'portfolio_no_zero_weightage.json'

with open(json_file_path, 'w') as json_file:
    json.dump(portfolio_no_zero_weightage, json_file)

In [47]:
results_stocks_intersection_of_both = {}
portfolio_intersection_of_both = {}

for year_month, stocks in yearWise_dict_stocks_intersection_of_both.items():

    year = year_month[:4]

    historical_price = final_stock_df[stocks]
    historical_price = historical_price[(historical_price.index >= f'{month}-01-{int(year)-5}') & (historical_price.index < f'{month}-01-{year}')]

    nifty50_temp = nifty50[(nifty50.index >= f'{month}-01-{int(year)-5}') & (nifty50.index < f'{month}-01-{year}')]
    nifty50_temp_wr = nifty50_temp.pct_change().dropna()

    weekly_returns = historical_price.pct_change().dropna()
    cov_matrix = weekly_returns.cov()

    symbols = historical_price.columns.to_list()

    results = {}
    stock_list = symbols.copy()

    for stock in stock_list:
        results[stock] = regress_stock_with_index(weekly_returns, stock, nifty50_temp_wr)

    alpha_beta_dict = {}
    alpha_list = []
    beta_list = []

    for stock in stock_list:
        alpha = results[stock]['const']
        beta = results[stock][('Nifty 50',)]
        alpha_list.append(alpha)
        beta_list.append(beta)
        alpha_beta_tuple = (alpha, beta)
        alpha_beta_dict[stock] = alpha_beta_tuple

    index_std_dev = nifty50_temp_wr.std().values[0]
    index_average_return = nifty50_temp_wr.mean().values[0]

    num_symbs = len(historical_price.columns)

    results_dict = smi_min_variance(index_average_return, index_std_dev, alpha_list, beta_list, expected_returns, num_symbs)

    results_stocks_intersection_of_both[year_month] = results_dict

    portfolio = []
    for i in range(len(symbols)):
        symbol = symbols[i]
        weight = results_dict['Optimal Weights'][i]
        portfolio.append((symbol, weight))

    portfolio_intersection_of_both[year_month] = portfolio

    symbols = top_8_stocks(portfolio_intersection_of_both)

    historical_price = final_stock_df[symbols]
    historical_price = historical_price[(historical_price.index >= f'{month}-01-{int(year)-5}') & (historical_price.index < f'{month}-01-{year}')]

    weekly_returns = historical_price.pct_change().dropna()
    cov_matrix = weekly_returns.cov()

    symbols = historical_price.columns.to_list()

    results = {}
    stock_list = symbols.copy()

    for stock in stock_list:
        results[stock] = regress_stock_with_index(weekly_returns, stock, nifty50_temp_wr)

    alpha_beta_dict = {}
    alpha_list = []
    beta_list = []

    for stock in stock_list:
        alpha = results[stock]['const']
        beta = results[stock][('Nifty 50',)]
        alpha_list.append(alpha)
        beta_list.append(beta)
        alpha_beta_tuple = (alpha, beta)
        alpha_beta_dict[stock] = alpha_beta_tuple

    index_std_dev = nifty50_temp_wr.std().values[0]
    index_average_return = nifty50_temp_wr.mean().values[0]

    num_symbs = len(historical_price.columns)

    results_dict = smi_max_returns(index_average_return, index_std_dev, alpha_list, beta_list, maximum_risk, num_symbs)

    results_stocks_intersection_of_both[year_month] = results_dict

    portfolio = []
    for i in range(len(symbols)):
        symbol = symbols[i]
        weight = results_dict['Optimal Weights'][i]
        portfolio.append((symbol, weight))

    portfolio_intersection_of_both[year_month] = portfolio

In [48]:
portfolio_intersection_of_both

{'2023_12': [('BSE.NS', 0.2499),
  ('SUZLON.NS', 0.1462),
  ('ORIENTCEM.NS', 0.0611),
  ('NEULANDLAB.NS', 0.0513),
  ('HSCL.NS', 0.0724),
  ('NEWGEN.NS', 0.0912),
  ('JPPOWER.NS', 0.078),
  ('GMDCLTD.NS', 0.2499)]}

In [49]:
json_file_path = 'portfolio_intersection_of_both.json'

with open(json_file_path, 'w') as json_file:
    json.dump(portfolio_intersection_of_both, json_file)

In [50]:
json_file_path = 'results_stocks_intersection_of_both.json'

with open(json_file_path, 'w') as json_file:
    json.dump(results_stocks_intersection_of_both, json_file)

In [51]:
json_file_path = 'results_stocks_no_zero_weightage.json'

with open(json_file_path, 'w') as json_file:
    json.dump(results_stocks_no_zero_weightage, json_file)

In [52]:
json_file_path = 'results_stocks_overall_highest_weightage.json'

with open(json_file_path, 'w') as json_file:
    json.dump(results_stocks_overall_highest_weightage, json_file)

In [53]:
results_stocks_overall_highest_weightage

{'2023_12': {'Optimal Weights': [0.0345,
   0.2499,
   0.0679,
   0.2499,
   0.1406,
   0.0682,
   0.0925,
   0.0965],
  'Min Variance': -0.009880195102806685,
  'Weekly Return': 0.00987932731314999,
  'Annual Return': 0.6672973830038105,
  'Weekly Std Dev': 0.004395729362674313,
  'Annual Std Dev': 0.03169805522036975}}

In [54]:
annual_return = 0
annual_variance = 0
annual_std_dev = 0

for year_month, stat_dict in results_stocks_overall_highest_weightage.items():
    annual_return += stat_dict['Annual Return']
    annual_variance += stat_dict['Min Variance']*52
    annual_std_dev += stat_dict['Annual Std Dev']

avg_annual_return = annual_return/61
avg_annual_variance = annual_variance/61
avg_annual_std_dev = annual_std_dev/61

avg_annual_return, avg_annual_variance, avg_annual_std_dev

(0.010939301360718204, -0.008422461399113894, 0.0005196402495142582)

In [55]:
annual_return = 0
annual_variance = 0
annual_std_dev = 0

for year_month, stat_dict in results_stocks_no_zero_weightage.items():
    annual_return += stat_dict['Annual Return']
    annual_variance += stat_dict['Min Variance']*52
    annual_std_dev += stat_dict['Annual Std Dev']

avg_annual_return = annual_return/61
avg_annual_variance = annual_variance/61
avg_annual_std_dev = annual_std_dev/61

avg_annual_return, avg_annual_variance, avg_annual_std_dev

(0.010939301360718204, -0.008422461399113894, 0.0005196402495142582)

In [56]:
annual_return = 0
annual_variance = 0
annual_std_dev = 0

for year_month, stat_dict in results_stocks_intersection_of_both.items():
    annual_return += stat_dict['Annual Return']
    annual_variance += stat_dict['Min Variance']*52
    annual_std_dev += stat_dict['Annual Std Dev']

avg_annual_return = annual_return/61
avg_annual_variance = annual_variance/61
avg_annual_std_dev = annual_std_dev/61

avg_annual_return, avg_annual_variance, avg_annual_std_dev

(0.011206443232198385, -0.00858468931163813, 0.000897827633674415)